In [ ]:
!pip install pyvi
!pip install transformers

#Import thư viện

In [ ]:
import pandas as pd
import numpy as np
import torch
from pyvi import ViTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from transformers import AutoModel, AutoTokenizer

#Lấy data cần thiết

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hotel_data = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_info_hotel_new.csv')

In [ ]:
# Đọc dữ liệu từ file CSV
df_his = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/dataset/data_history.csv')
df_hotel = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/dataset/data_hotel.csv')

# Chuyển đổi cột 'Date' thành kiểu dữ liệu datetime
df_his['Date'] = pd.to_datetime(df_his['Date'])

# Sắp xếp DataFrame theo cột 'Date' theo thứ tự giảm dần
df_his = df_his.sort_values(by='Date', ascending=False)

# Lấy dữ liệu lịch sử gần nhất cho mỗi người dùng
df_latest = df_his.groupby('IDuser').first().reset_index()
df_latest = pd.merge(df_latest, df_hotel, on = 'IDhotel')
df_latest = pd.merge(df_latest, hotel_data, on = 'NameHotel')

df_latest

,IDuser,IDhotel,Rating,Date,NameHotel,Location_x,Unnamed: 0,Location_y,RatingHotel,CountRating,Price,Facilities,Quality,DistanceCenter,Around,Vicinity,Link,Price_Category
0,1,580,8.0,2022-06-26,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",0,1.700,"Tượng đài, Cafe/quán bar, Biển","Di tích, Khu cắm trại, Công viên, Biển, Sân ba...",https://www.booking.com/hotel/vn/duc-trong-tha...,Thấp
1,99,580,9.0,2022-07-08,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",0,1.700,"Tượng đài, Cafe/quán bar, Biển","Di tích, Khu cắm trại, Công viên, Biển, Sân ba...",https://www.booking.com/hotel/vn/duc-trong-tha...,Thấp
2,112,580,8.0,2023-08-07,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",0,1.700,"Tượng đài, Cafe/quán bar, Biển","Di tích, Khu cắm trại, Công viên, Biển, Sân ba...",https://www.booking.com/hotel/vn/duc-trong-tha...,Thấp
3,333,580,6.0,2022-08-25,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",0,1.700,"Tượng đài, Cafe/quán bar, Biển","Di tích, Khu cắm trại, Công viên, Biển, Sân ba...",https://www.booking.com/hotel/vn/duc-trong-tha...,Thấp
4,783,580,5.0,2022-09-13,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",0,1.700,"Tượng đài, Cafe/quán bar, Biển","Di tích, Khu cắm trại, Công viên, Biển, Sân ba...",https://www.booking.com/hotel/vn/duc-trong-tha...,Thấp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8975,8357,65,8.8,2018-08-13,Famiana Resort & Spa Phú Quốc,Phú Quốc,277,Phú Quốc,8.9,211,1476090,"Máy lạnh, Nhà hàng, Hồ bơi, Lễ tân 24h, WiFi",4,0.619,Biển,"Sân bay, Chùa, Di tích, Miếu, Sân vận động, Su...",https://www.traveloka.com/vi-vn/hotel/detail?s...,Cao
8976,8444,65,8.5,2023-04-19,Famiana Resort & Spa Phú Quốc,Phú Quốc,277,Phú Quốc,8.9,211,1476090,"Máy lạnh, Nhà hàng, Hồ bơi, Lễ tân 24h, WiFi",4,0.619,Biển,"Sân bay, Chùa, Di tích, Miếu, Sân vận động, Su...",https://www.traveloka.com/vi-vn/hotel/detail?s...,Cao
8977,8772,88,8.0,2022-08-20,Hoi An Golden Holiday Hotel & Spa,Hội An,168,Hội An,9.8,1800,1080000,"Hồ bơi ngoài trời, Xe đưa đón sân bay, Phòng...",4,1.200,"Bảo tàng, Cafe/quán bar","Chùa, Bảo tàng, Di tích, Núi, Sông, Biển, Sân bay",https://www.booking.com/hotel/vn/golden-holida...,Cao
8978,9085,400,9.1,2016-10-09,Lotus Rock Hotel,Đà Nẵng,217,Đà Nẵng,8.2,64,506000,"Máy lạnh, Thang máy, WiFi",1,0.441,"Công viên, Chùa, Bệnh viện","Sân bóng đá, Bảo tàng, Cầu, Siêu thị, Khu giải...",https://www.traveloka.com/vi-vn/hotel/detail?s...,Trung bình


#Xử lí dữ liệu đầu vào

In [ ]:
def preprocess_text(text):
    # Remove commas
    text = text.replace(',', '')

    # Tokenize the text using pyvi
    tokens = ViTokenizer.tokenize(text)

    # Remove stopwords and convert to lowercase
    tokens = tokens.lower().split()

    # Join tokens back into a text
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
# Áp dụng hàm preprocess_text vào 4 cột Text1, Text2, Text3, Text4
text_columns = ['Facilities', 'Around', 'Vicinity', 'Price_Category']
for column in text_columns:
    df_latest[column] = df_latest[column].apply(preprocess_text)

,IDuser,IDhotel,Rating,Date,NameHotel,Location_x,Unnamed: 0,Location_y,RatingHotel,CountRating,Price,Facilities,Quality,DistanceCenter,Around,Vicinity,Link,Price_Category
0,1,580,8.0,2022-06-26,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,wifi miễn_phí chỗ đỗ xe miễn_phí phòng gia_đình,0,1.700,tượng_đài cafe / quán bar biển,di_tích khu_cắm trại công_viên biển sân_bay hả...,https://www.booking.com/hotel/vn/duc-trong-tha...,thấp
1,99,580,9.0,2022-07-08,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,wifi miễn_phí chỗ đỗ xe miễn_phí phòng gia_đình,0,1.700,tượng_đài cafe / quán bar biển,di_tích khu_cắm trại công_viên biển sân_bay hả...,https://www.booking.com/hotel/vn/duc-trong-tha...,thấp
2,112,580,8.0,2023-08-07,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,wifi miễn_phí chỗ đỗ xe miễn_phí phòng gia_đình,0,1.700,tượng_đài cafe / quán bar biển,di_tích khu_cắm trại công_viên biển sân_bay hả...,https://www.booking.com/hotel/vn/duc-trong-tha...,thấp
3,333,580,6.0,2022-08-25,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,wifi miễn_phí chỗ đỗ xe miễn_phí phòng gia_đình,0,1.700,tượng_đài cafe / quán bar biển,di_tích khu_cắm trại công_viên biển sân_bay hả...,https://www.booking.com/hotel/vn/duc-trong-tha...,thấp
4,783,580,5.0,2022-09-13,Đức Trọng Hotel,Vũng Tàu,71,Vũng Tàu,7.8,854,147623,wifi miễn_phí chỗ đỗ xe miễn_phí phòng gia_đình,0,1.700,tượng_đài cafe / quán bar biển,di_tích khu_cắm trại công_viên biển sân_bay hả...,https://www.booking.com/hotel/vn/duc-trong-tha...,thấp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8975,8357,65,8.8,2018-08-13,Famiana Resort & Spa Phú Quốc,Phú Quốc,277,Phú Quốc,8.9,211,1476090,máy_lạnh nhà_hàng hồ bơi lễ_tân 24h wifi,4,0.619,biển,sân_bay chùa_di_tích miếu sân vận_động suối th...,https://www.traveloka.com/vi-vn/hotel/detail?s...,cao
8976,8444,65,8.5,2023-04-19,Famiana Resort & Spa Phú Quốc,Phú Quốc,277,Phú Quốc,8.9,211,1476090,máy_lạnh nhà_hàng hồ bơi lễ_tân 24h wifi,4,0.619,biển,sân_bay chùa_di_tích miếu sân vận_động suối th...,https://www.traveloka.com/vi-vn/hotel/detail?s...,cao
8977,8772,88,8.0,2022-08-20,Hoi An Golden Holiday Hotel & Spa,Hội An,168,Hội An,9.8,1800,1080000,hồ bơi ngoài_trời xe đưa_đón sân_bay phòng_khô...,4,1.200,bảo_tàng cafe / quán bar,chùa bảo_tàng di_tích núi_sông biển sân_bay,https://www.booking.com/hotel/vn/golden-holida...,cao
8978,9085,400,9.1,2016-10-09,Lotus Rock Hotel,Đà Nẵng,217,Đà Nẵng,8.2,64,506000,máy_lạnh thang_máy wifi,1,0.441,công_viên chùa bệnh_viện,sân bóng_đá bảo_tàng cầu_siêu_thị khu giải_trí...,https://www.traveloka.com/vi-vn/hotel/detail?s...,trung_bình


In [ ]:
# Áp dụng hàm preprocess_text vào 4 cột Text1, Text2, Text3, Text4
text_columns = ['Facilities', 'Around', 'Vicinity', 'Price_Category']
for column in text_columns:
    hotel_data[column] = hotel_data[column].apply(preprocess_text)

#Trích xuất đặc trưng với phoBERT

In [ ]:
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base") # vinai/phobert-large
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)# vinai/phobert-large
    return v_phobert, v_tokenizer
print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
print("Đã nạp xong model BERT.")

Chuẩn bị nạp model BERT....


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Đã nạp xong model BERT.


In [ ]:
# Tạo danh sách để lưu trữ các vecto đặc trưng
fac_features = []
ar_features = []
vici_features = []
pri_features = []

In [ ]:
# Tạo danh sách để lưu trữ các vecto đặc trưng
hfac_features = []
har_features = []
hvici_features = []
hpri_features = []

In [ ]:
# Tính toán độ dài tối đa của câu văn trong 4 cột
max_length = 0
for column in ['Facilities', 'Around', 'Vicinity', 'Price_Category']:  # Thay thế 'column1', 'column2',... bằng tên thật của các cột trong dataset của bạn
    column_lengths = df_latest[column].apply(lambda x: len(tokenizer.encode(x)))  # Tính độ dài của câu văn bằng tokenizer
    max_length = max(max_length, column_lengths.max())

##Trích xuất đặc trưng của khách sạn

In [ ]:
# Duyệt qua từng dòng trong dataset
for index, row in hotel_data.iterrows():
    # Lấy nội dung của từng cột trong dòng hiện tại
    column1_sentence = row['Facilities']
    column2_sentence = row['Around']
    column3_sentence = row['Vicinity']
    column4_sentence = row['Price_Category']

    # Mã hóa câu văn bằng tokenizer
    column1_input_ids = torch.tensor([tokenizer.encode(column1_sentence)])
    column2_input_ids = torch.tensor([tokenizer.encode(column2_sentence)])
    column3_input_ids = torch.tensor([tokenizer.encode(column3_sentence)])
    column4_input_ids = torch.tensor([tokenizer.encode(column4_sentence)])

    # Mã hóa câu văn bằng mô hình PhoBERT
    with torch.no_grad():
        column1_output = phobert(column1_input_ids)
        column2_output = phobert(column2_input_ids)
        column3_output = phobert(column3_input_ids)
        column4_output = phobert(column4_input_ids)

    # Lấy vector đặc trưng từ đầu ra mô hình
    column1_feature = column1_output[0][0].numpy()
    column2_feature = column2_output[0][0].numpy()
    column3_feature = column3_output[0][0].numpy()
    column4_feature = column4_output[0][0].numpy()

    # Điều chỉnh kích thước của các vector đặc trưng để có cùng shape
    column1_feature = np.pad(column1_feature, ((0, max_length - len(column1_feature)), (0, 0)), mode='constant')
    column2_feature = np.pad(column2_feature, ((0, max_length - len(column2_feature)), (0, 0)), mode='constant')
    column3_feature = np.pad(column3_feature, ((0, max_length - len(column3_feature)), (0, 0)), mode='constant')
    column4_feature = np.pad(column4_feature, ((0, max_length - len(column4_feature)), (0, 0)), mode='constant')

    # Lưu trữ vector đặc trưng vào từng danh sách tương ứng
    hfac_features.append(column1_feature)
    har_features.append(column2_feature)
    hvici_features.append(column3_feature)
    hpri_features.append(column4_feature)

##Trích xuất đặc trưng của người dùng

In [ ]:
# Duyệt qua từng dòng trong dataset
for index, row in df_latest.iterrows():
    # Lấy nội dung của từng cột trong dòng hiện tại
    # column1_sentence = row['Facilities']
    # column2_sentence = row['Around']
    # column3_sentence = row['Vicinity']
    column4_sentence = row['Price_Category']

    # Mã hóa câu văn bằng tokenizer
    # column1_input_ids = torch.tensor([tokenizer.encode(column1_sentence)])
    # column2_input_ids = torch.tensor([tokenizer.encode(column2_sentence)])
    # column3_input_ids = torch.tensor([tokenizer.encode(column3_sentence)])
    column4_input_ids = torch.tensor([tokenizer.encode(column4_sentence)])

    # Mã hóa câu văn bằng mô hình PhoBERT
    with torch.no_grad():
        # column1_output = phobert(column1_input_ids)
        # column2_output = phobert(column2_input_ids)
        # column3_output = phobert(column3_input_ids)
        column4_output = phobert(column4_input_ids)

    # Lấy vector đặc trưng từ đầu ra mô hình
    # column1_feature = column1_output[0][0].numpy()
    # column2_feature = column2_output[0][0].numpy()
    # column3_feature = column3_output[0][0].numpy()
    column4_feature = column4_output[0][0].numpy()

    # Điều chỉnh kích thước của các vector đặc trưng để có cùng shape
    # column1_feature = np.pad(column1_feature, ((0, max_length - len(column1_feature)), (0, 0)), mode='constant')
    # column2_feature = np.pad(column2_feature, ((0, max_length - len(column2_feature)), (0, 0)), mode='constant')
    # column3_feature = np.pad(column3_feature, ((0, max_length - len(column3_feature)), (0, 0)), mode='constant')
    column4_feature = np.pad(column4_feature, ((0, max_length - len(column4_feature)), (0, 0)), mode='constant')

    # Lưu trữ vector đặc trưng vào từng danh sách tương ứng
    # fac_features.append(column1_feature)
    # ar_features.append(column2_feature)
    # vici_features.append(column3_feature)
    pri_features.append(column4_feature)

#Xuất các đặc trưng dưới dạng CSV

In [ ]:
import csv
import numpy as np

# Danh sách chứa 8000 mảng
array_list = hvici_features  # Thay [...]) bằng danh sách chứa 8000 mảng của bạn

# Đường dẫn tệp CSV đích
csv_file_path = 'hvici_features.csv'

# Ghi mảng vào tệp CSV
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for array in array_list:
        writer.writerow(array.flatten())

print("Danh sách đã được chuyển sang tệp CSV thành công.")

Danh sách đã được chuyển sang tệp CSV thành công.
